# Another pointless comparison of binary size between Rust and C++

I recently saw a Youtube video of someone going through a laundry list of complaints about Rust. The merits of the discussion notwithstanding, one of the points raised was about the difference in binary size produced from different compilers.

There are [many](https://github.com/johnthagen/min-sized-rust), [many](https://os.phil-opp.com/freestanding-rust-binary/), [many](https://lifthrasiir.github.io/rustlog/why-is-a-rust-executable-large.html), [many](https://kripken.github.io/blog/binaryen/2018/04/18/rust-emscripten.html), [many](https://stackoverflow.com/questions/29008127/why-are-rust-executables-so-huge), [many](https://github.com/rust-embedded/wg/issues/5) discussions about Rust binary sizes online and I'm not going to rehash any of that. I was just curious about reproducing the data in the Youtube video, and in particular I am generally interested in producing static binaries for simplified deployment reasons.  I realise the static-vs-dynamic debate is a whole other flamewar online and I don't really care 😃

_This post was made in a Jupyter Notebook using the awesome [evcxr](https://github.com/google/evcxr) Rust interpreter. All the cells were executed live in the notebook._

## Utility functions

In [2]:
/// Tool to easily run shell commands.
fn sh(dir: &str, cmd: &str, args: &[&'static str]) -> Result<(), Box<dyn std::error::Error>> {
    let output = std::process::Command::new(cmd)
        .current_dir(dir)
        .args(args)
        .output()?;
    if output.stdout.len() > 0 {
        println!("stdout:\n{}", String::from_utf8(output.stdout)?);
    }
    if output.stderr.len() > 0 {
        println!("stderr:\n{}", String::from_utf8(output.stderr)?);
    }
    Ok(())
}

# C++ binary size

The example given in the video was like this:

In [3]:
std::fs::remove_dir_all("~/tmp/cppsize/").ok();
std::fs::create_dir_all("~/tmp/cppsize/")?;
std::fs::write("~/tmp/cppsize/main.cc", "int main() {}")?;

In [4]:
sh("~/tmp/cppsize", "ls", &vec!["-lah"])?;

stdout:
total 4.0K
drwxr-xr-x. 1 caleb caleb 14 Jun  1 12:20 .
drwxr-xr-x. 1 caleb caleb 14 Jun  1 12:20 ..
-rw-r--r--. 1 caleb caleb 13 Jun  1 12:20 main.cc



Compiling the empty file, according to the aforementioned video:

In [5]:
sh(
    "~/tmp/cppsize",
    "g++",
    &vec!["-o", "main", 
        "main.cc", 
        "-Ofast", 
        "-std=c++20", 
        "-s", 
        "-flto", 
        "-static-libgcc", 
        "-static-libstdc++"
    ]
)?;

In [6]:
sh("~/tmp/cppsize", "ls", &vec!["-lah"])?;

stdout:
total 16K
drwxr-xr-x. 1 caleb caleb  22 Jun  1 12:20 .
drwxr-xr-x. 1 caleb caleb  14 Jun  1 12:20 ..
-rwxr-xr-x. 1 caleb caleb 11K Jun  1 12:20 main
-rw-r--r--. 1 caleb caleb  13 Jun  1 12:20 main.cc



In the Youtube video, the very low binary size of `main` was as given above, around **11 kB**. We can look at the runtime dynamic dependencies with `ldd`:

In [7]:
sh("~/tmp/cppsize", "ldd", &vec!["main"])?;

stdout:
	linux-vdso.so.1 (0x00007f98cdc15000)
	libm.so.6 => /lib64/libm.so.6 (0x00007f98cdb02000)
	libc.so.6 => /lib64/libc.so.6 (0x00007f98cd910000)
	/lib64/ld-linux-x86-64.so.2 (0x00007f98cdc17000)



We want to make comparisons without being affected by which dependencies are being dynamically linked and which are statically linked. So, we can add one small extra option to the compiler options for this empty C++ example to produce a fully static binary.

You may need to install static versions of some libraries on your system for the next command to work. On Fedora Linux, this command would be:

```shell
sudo dnf install glibc-static libstdc++-static
```

Here is the static compile:

In [12]:
sh(
    "~/tmp/cppsize",
    "g++",
    &vec!["-o", "main", 
        "main.cc", 
        "-Ofast", 
        "-std=c++20", 
        "-s", 
        "-flto", 
        "-static-libgcc", 
        "-static-libstdc++",
        "-static"  // <--- THIS IS THE EXTRA OPTION
    ]
)?;

In [13]:
sh("~/tmp/cppsize", "ls", &vec!["-lah"])?;

stdout:
total 708K
drwxr-xr-x. 1 caleb caleb   22 Jun  1 12:32 .
drwxr-xr-x. 1 caleb caleb   14 Jun  1 12:20 ..
-rwxr-xr-x. 1 caleb caleb 701K Jun  1 12:32 main
-rw-r--r--. 1 caleb caleb   13 Jun  1 12:20 main.cc



Now the size is much bigger, around **701 kB**. `ldd` tells us the binary no longer links dynamically to anything:

In [14]:
sh("~/tmp/cppsize", "ldd", &vec!["main"])?;

stderr:
	not a dynamic executable



## Rust binary size

In the Youtube video, an empty rust file was compiled in the following way:

In [15]:
std::fs::remove_dir_all("~/tmp/rustsize/").ok();
std::fs::create_dir_all("~/tmp/rustsize/")?;
std::fs::write("~/tmp/rustsize/a.rs", "fn main() {}")?;

In [16]:
sh("~/tmp/rustsize", "ls", &vec!["-lah"])?;

stdout:
total 4.0K
drwxr-xr-x. 1 caleb caleb  8 Jun  1 12:32 .
drwxr-xr-x. 1 caleb caleb 30 Jun  1 12:32 ..
-rw-r--r--. 1 caleb caleb 12 Jun  1 12:32 a.rs



For some reason the video author didn't want to use Cargo 🤷. Anyway let's go with it:

In [17]:
sh(
    "~/tmp/rustsize",
    "rustc",
    &vec![
        "-O", 
        "-C", "strip=symbols",
        "a.rs", 
    ]
)?;

In [18]:
sh("~/tmp/rustsize", "ls", &vec!["-lah"])?;

stdout:
total 352K
drwxr-xr-x. 1 caleb caleb   10 Jun  1 12:33 .
drwxr-xr-x. 1 caleb caleb   30 Jun  1 12:32 ..
-rwxr-xr-x. 1 caleb caleb 346K Jun  1 12:33 a
-rw-r--r--. 1 caleb caleb   12 Jun  1 12:32 a.rs



Indeed, we find a similar number as the author in the video, around **350 kB**. The author concludes that Rust-induced bloat is 346/11 => 30X bigger.

Let's add some optimizations, and work towards a static build to do the comparison correctly:

In [19]:
sh(
    "~/tmp/rustsize",
    "rustc",
    &vec![
        "-O", 
        "-C", "strip=symbols",
        "-C", "lto=on",
        "-C", "codegen-units=1",
        "a.rs", 
    ]
)?;

In [20]:
sh("~/tmp/rustsize", "ls", &vec!["-lah"])?;

stdout:
total 324K
drwxr-xr-x. 1 caleb caleb   10 Jun  1 12:36 .
drwxr-xr-x. 1 caleb caleb   30 Jun  1 12:32 ..
-rwxr-xr-x. 1 caleb caleb 318K Jun  1 12:36 a
-rw-r--r--. 1 caleb caleb   12 Jun  1 12:32 a.rs



Marginal improvement.

There are a few more extra options beyond what was used in the C++ example, but these don't make a significant difference to a completely-empty program.

In [21]:
sh(
    "~/tmp/rustsize",
    "rustc",
    &vec![
        "-O", 
        "-C", "strip=symbols",
        "-C", "lto=on",
        "-C", "codegen-units=1",
        
        // Extra
        "-C", "opt-level=s",  // Optimize for size
        "-C", "panic=abort",  // Disable stack-unwinding
        
        "a.rs", 
    ]
)?;

In [22]:
sh("~/tmp/rustsize", "ls", &vec!["-lah"])?;

stdout:
total 304K
drwxr-xr-x. 1 caleb caleb   10 Jun  1 12:36 .
drwxr-xr-x. 1 caleb caleb   30 Jun  1 12:32 ..
-rwxr-xr-x. 1 caleb caleb 298K Jun  1 12:36 a
-rw-r--r--. 1 caleb caleb   12 Jun  1 12:32 a.rs



These extra flags and settings are making marginal improvements. We haven't yet checked whether the Rust binary is linking dynamically to anything yet.  Apples-to-apples, remember? So let's look into that:

In [23]:
sh("~/tmp/rustsize", "ldd", &vec!["a"])?;

stdout:
	linux-vdso.so.1 (0x00007fd2149ca000)
	libgcc_s.so.1 => /lib64/libgcc_s.so.1 (0x00007fd21492e000)
	libc.so.6 => /lib64/libc.so.6 (0x00007fd21473c000)
	/lib64/ld-linux-x86-64.so.2 (0x00007fd2149cc000)



Woah, the Rust binary is dynamically linked to libc! Why does it need to do that?  Here's a great concise explanation [from 2015](https://news.ycombinator.com/item?id=9436004):

> erickt on April 24, 2015:
>
> No it's not a stupid question. libc (or CRT on windows) really is the library that exposes all the user space system libraries. It contains the functions to do IO, sockets, threads, and etc. So we use it to expose that functionality to rust users.
>
> Now there are some languages, namely Go, that skip libc and just implement directly against the syscall interface. Go has the advantage of being able to draw from Google's vast experience interacting deep within the system, so it was comparatively cheap for them to do this.
>
> For rust, it never really felt like it was worth the effort for the benefit we'd get out of it. It was more important to get the language done. 

So Rust is still using the C runtime to interact with the OS through userspace libraries provided in the C runtime. So it doesn't need the C or C++ standard libraries, only the runtime for OS interaction.

Ok so for a fair comparison we should compare the sizes after statically linking against the CRT.


In [24]:
sh(
    "~/tmp/rustsize",
    "rustc",
    &vec![
        "-O", 
        "-C", "strip=symbols",
        "-C", "lto=on",
        "-C", "codegen-units=1",
        "-C", "opt-level=s",  // Optimize for size
        "-C", "panic=abort",  // Disable stack-unwinding
        
        // New
        "-C", "target-feature=+crt-static",
        
        "a.rs", 
    ]
)?;

In [25]:
sh("~/tmp/rustsize", "ldd", &vec!["a"])?;

stdout:
	statically linked



And the size?

In [26]:
sh("~/tmp/rustsize", "ls", &vec!["-lah"])?;

stdout:
total 1.2M
drwxr-xr-x. 1 caleb caleb   10 Jun  1 12:39 .
drwxr-xr-x. 1 caleb caleb   30 Jun  1 12:32 ..
-rwxr-xr-x. 1 caleb caleb 1.2M Jun  1 12:39 a
-rw-r--r--. 1 caleb caleb   12 Jun  1 12:32 a.rs



Now it's a whopping 1.2 MB. So if we do all the work to produce a fully static executable from an empty `main()` for both Rust and C++, we indeed find that C++ produces the smaller binary. It's 1 - 721/1200 => **40%** smaller than the Rust binary, which is clearly a much smaller difference than the ~ 30X comparison I gave at the start.

# What happens when the program is not empty?

What happens to the comparison if the `main()` function is not empty, but instead uses some stdlib functionality?

For C++:

In [51]:
std::fs::remove_dir_all("~/tmp/cppsize3/").ok();
std::fs::create_dir_all("~/tmp/cppsize3/")?;
let content = r#"
#include <cstdio>
int main() {
    printf("Hello world\n");
    return 0;
}
"#;
std::fs::write("~/tmp/cppsize3/main.cc", content)?;

In [52]:
sh(
    "~/tmp/cppsize3",
    "g++",
    &vec!["-o", "main", 
        "main.cc", 
        "-Ofast", 
        "-std=c++20", 
        "-s", 
        "-flto", 
        "-static-libgcc", 
        "-static-libstdc++",
        "-static"  // <--- THIS IS THE EXTRA OPTION
    ]
)?;

In [53]:
sh("~/tmp/cppsize3", "strip", &vec!["main"])?;

In [54]:
sh("~/tmp/cppsize3", "ls", &vec!["-lah"])?;

stdout:
total 700K
drwxr-xr-x. 1 caleb caleb   22 Jun  1 13:02 .
drwxr-xr-x. 1 caleb caleb   80 Jun  1 13:02 ..
-rwxr-xr-x. 1 caleb caleb 694K Jun  1 13:02 main
-rw-r--r--. 1 caleb caleb   77 Jun  1 13:02 main.cc



And let's also check using the C++ `iostream` stdlib:

In [55]:
std::fs::remove_dir_all("~/tmp/cppsize2/").ok();
std::fs::create_dir_all("~/tmp/cppsize2/")?;
let content = r#"
#include <iostream>
int main() {
    std::cout << "Hello world" << std::endl;
    return 0;
}
"#;
std::fs::write("~/tmp/cppsize2/main.cc", content)?;

In [56]:
sh(
    "~/tmp/cppsize2",
    "g++",
    &vec!["-o", "main", 
        "main.cc", 
        "-Ofast", 
        "-std=c++20", 
        "-s", 
        "-flto", 
        "-static-libgcc", 
        "-static-libstdc++",
        "-static"  // <--- THIS IS THE EXTRA OPTION
    ]
)?;

In [57]:
sh("~/tmp/cppsize2", "strip", &vec!["main"])?;

In [58]:
sh("~/tmp/cppsize2", "ls", &vec!["-lah"])?;

stdout:
total 1.9M
drwxr-xr-x. 1 caleb caleb   22 Jun  1 13:03 .
drwxr-xr-x. 1 caleb caleb   80 Jun  1 13:03 ..
-rwxr-xr-x. 1 caleb caleb 1.8M Jun  1 13:03 main
-rw-r--r--. 1 caleb caleb   95 Jun  1 13:03 main.cc



For Rust:

In [64]:
std::fs::remove_dir_all("~/tmp/rustsize3/").ok();
std::fs::create_dir_all("~/tmp/rustsize3/")?;
let content = r#"
use std::io::{self, Write};

fn main() -> io::Result<()> {
    io::stdout().write_all(b"Hello world\n")?;
    Ok(())
}
"#;
std::fs::write("~/tmp/rustsize3/main.rs", content)?;

In [65]:
sh(
    "~/tmp/rustsize3",
    "rustc",
    &vec![
        "-O", 
        "-C", "strip=symbols",
        "-C", "lto=on",
        "-C", "codegen-units=1",
        "-C", "opt-level=s",  // Optimize for size
        "-C", "panic=abort",  // Disable stack-unwinding
        
        // New
        "-C", "target-feature=+crt-static",
        
        "main.rs", 
    ]
)?;

In [66]:
sh("~/tmp/rustsize3", "ls", &vec!["-lah"])?;

stdout:
total 1.2M
drwxr-xr-x. 1 caleb caleb   22 Jun  1 13:07 .
drwxr-xr-x. 1 caleb caleb   98 Jun  1 13:07 ..
-rwxr-xr-x. 1 caleb caleb 1.2M Jun  1 13:07 main
-rw-r--r--. 1 caleb caleb  120 Jun  1 13:07 main.rs



And let's also check the `println!()` macro:

In [48]:
std::fs::remove_dir_all("~/tmp/rustsize2/").ok();
std::fs::create_dir_all("~/tmp/rustsize2/")?;
let content = r#"
fn main() {
    println!("Hello world");
}
"#;
std::fs::write("~/tmp/rustsize2/main.rs", content)?;

In [49]:
sh(
    "~/tmp/rustsize2",
    "rustc",
    &vec![
        "-O", 
        "-C", "strip=symbols",
        "-C", "lto=on",
        "-C", "codegen-units=1",
        "-C", "opt-level=s",  // Optimize for size
        "-C", "panic=abort",  // Disable stack-unwinding
        
        // New
        "-C", "target-feature=+crt-static",
        
        "main.rs", 
    ]
)?;

In [50]:
sh("~/tmp/rustsize2", "ls", &vec!["-lah"])?;

stdout:
total 1.2M
drwxr-xr-x. 1 caleb caleb   22 Jun  1 12:51 .
drwxr-xr-x. 1 caleb caleb   64 Jun  1 12:51 ..
-rwxr-xr-x. 1 caleb caleb 1.2M Jun  1 12:51 main
-rw-r--r--. 1 caleb caleb   44 Jun  1 12:51 main.rs



When the C++ code is using `printf`, the rust binary is still larger (1.2 MB vs 700 kb), but when the C++ code is using `iostream`, the rust hello-world binary is smaller than the C++ one, by 1.2/1.8 => 34%. (In rust there is no difference whether `println!` or `stdout().write_all()` is used).

## Conclusion

What do you think this is, a research paper? My parting words are that none of this really matters. Binary size is certainly a concern in certain environments, but the real impact of whether you use Rust, C or C++ is not going to matter. The people working in the embedded space are already all over these issues. If you're interested in the embedded domain, do go check out the [Embedded Rust Working Group](http://blog.rust-embedded.org/).